In [1]:
from pathlib import Path
from typing import Tuple, Union, List, Dict, Iterable, Optional

import torch
import numpy as np
from tqdm.notebook import tqdm

from decomposer import Decomposer, DecomposerConfig
from recomposer import Recomposer, RecomposerConfig
from helpers import *

/home/webson/Research/congressional_adversary/congressional_env/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'decomposer.Decomposer' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Vocab size = 138,441


In [11]:
# base_dir = Path('../../results/news/validation')
base_dir = Path('../../results/news/validation')
deno_space = load_decomposers_en_masse(base_dir, patterns='*/epoch3.pt')
# deno_space.update(load_decomposers_en_masse(base_dir, patterns='*/epoch3.pt'))
print(deno_space.keys())

Loading ../../results/news/validation/-0.01d/epoch3.pt
Loading ../../results/news/validation/-3c/epoch3.pt
Loading ../../results/news/validation/-3c BS1024/epoch3.pt

dict_keys(['-0.01d/epoch3.pt', '-3c/epoch3.pt', '-3c BS1024/epoch3.pt'])


In [14]:
from evaluations.word_similarity.all_wordsim import mean_delta, read_plain_text, compare
import math

SELECT_MODEL = deno_space['-3c/epoch3.pt']

def convert(embed):
    word_vecs: Dict[str, np.array] = {}
    for word_id, vector in enumerate(embed):
        vector /= math.sqrt((vector ** 2).sum() + 1e-6)
        word_vecs[ITW[word_id]] = vector
    return word_vecs

def cf_WS(m1, m2):
    compare(convert(m1), convert(m2))

def cf(query):
    GD(query)
    print('Pretrained:')
    nearest_neighbors(query, PE)
    print('Our model:')
    nearest_neighbors(query, SELECT_MODEL)
    
cf_WS(SELECT_MODEL, PE)

             Dataset             Rho       Reference      Delta >.02
    EN-MTurk-771.txt         42.81%         52.84%         -10.03%
  EN-RW-STANFORD.txt         31.76%         45.33%         -13.58%
   EN-SIMLEX-999.txt         19.06%         32.92%         -13.86%
    EN-MTurk-287.txt         49.16%         59.11%          -9.95%
       EN-YP-130.txt         35.61%         50.72%         -15.11%
   EN-WS-353-SIM.txt         51.10%         68.51%         -17.41%
    EN-MEN-TR-3k.txt         49.28%         52.49%          -3.21%
   EN-WS-353-REL.txt         37.89%         44.70%          -6.81%
     EN-VERB-143.txt         15.61%         29.87%         -14.26%
   EN-WS-353-ALL.txt         42.31%         58.15%         -15.84%
 EN-SimVerb-3500.txt         10.11%         26.25%         -16.14%
Mean Delta = -12.3818%


In [24]:
GD('aca')

aca	0.6029, 0.0328, 0.1867, 0.0687, 0.1089, 	1140, 62, 353, 130, 206, 


In [15]:
cf('estate_tax')
cf('death_tax')

estate_tax	0.3422, 0.1765, 0.1551, 0.1604, 0.1658, 	128, 66, 58, 60, 62, 
Pretrained:
0.8058	death_tax
0.7939	inheritance_tax
0.7891	alternative_minimum
0.7830	surtax
0.7819	richest_americans
0.7713	top_tax
0.7479	individual_tax
0.7476	trillion_tax
0.7456	marriage_penalty
0.7389	top_rate

Our model:
0.3864	deductions
0.3733	capital_gains
0.3508	individual_tax
0.3486	alternative_minimum
0.3193	tax
0.3094	corporate_tax
0.3091	deduction
0.3030	tax_loopholes
0.3017	tax_rate
0.3005	getting_tough

death_tax	0.1286, 0.1429, 0.2714, 0.1143, 0.3429, 	9, 10, 19, 8, 24, 
Pretrained:
0.8740	inheritance_tax
0.8266	surtax
0.8179	estate_taxes
0.8133	wealthy_tax
0.8133	obamacare_taxes
0.8108	alternative_minimum
0.8090	cuts_expire
0.8088	personal_exemptions
0.8077	raises_taxes
0.8068	marriage_penalty

Our model:
0.3797	apostrophe
0.3774	good_solution
0.3696	generally_think
0.3643	allow_workers
0.3639	teacher_evaluations
0.3618	law_already
0.3532	business_taxes
0.3532	percent_bracket
0.3508	job_retraini

In [16]:
cf('undocumented_workers')
cf('illegal_aliens')
cf('chain_migration')

undocumented_workers	0.1814, 0.1535, 0.2326, 0.2512, 0.1814, 	39, 33, 50, 54, 39, 
Pretrained:
0.7386	unauthorized_workers
0.7354	work_visas
0.7099	hire_illegal
0.7078	million_undocumented
0.7017	temporary_work
0.7002	young_undocumented
0.6967	million_unauthorized
0.6966	work_authorization
0.6951	young_illegal
0.6937	called_dreamers

Our model:
0.3367	unauthorized_migrants
0.3178	temporary_visas
0.3158	alternative_development
0.3147	mexican_law
0.3113	immigrants_would
0.3071	law_since
0.3066	million_illegal
0.3058	rising_rates
0.3055	misbehaving
0.3047	white_says

illegal_aliens	0.0856, 0.0152, 0.1065, 0.0124, 0.7804, 	90, 16, 112, 13, 821, 
Pretrained:
0.7453	illegal_immigrants
0.7422	illegals
0.7150	undocumented_immigrants
0.7089	criminal_illegal
0.6989	criminal_aliens
0.6969	illegal_immigrant
0.6916	million_illegal
0.6883	young_illegal
0.6855	unauthorized_immigrants
0.6716	million_immigrants

Our model:
0.4160	illegal_immigrants
0.3978	illegals
0.3678	illegal_immigration
0.3585	amne

In [17]:
cf('obamacare')
cf('aca')
cf('public_option')
cf('single_payer')
cf('socialized_medicine')

obamacare	0.4485, 0.0143, 0.1767, 0.0278, 0.3327, 	4001, 128, 1576, 248, 2968, 
Pretrained:
0.8302	aca
0.8285	affordable_care
0.7863	health_law
0.7632	act_aca
0.7479	obamacare_insurance
0.7416	obamacare_would
0.7394	patient_protection
0.7364	republican_plan
0.7360	healthcare_law
0.7340	care_law

Our model:
0.6039	affordable_care
0.4987	aca
0.4463	repeal
0.4459	insurance
0.4427	health_insurance
0.4069	health_care
0.4016	act
0.3991	medicaid_expansion
0.3632	repealing
0.3596	subsidies

aca	0.6029, 0.0328, 0.1867, 0.0687, 0.1089, 	1140, 62, 353, 130, 206, 
Pretrained:
0.8302	obamacare
0.8112	affordable_care
0.7640	individual_mandate
0.7463	health_law
0.7154	act_aca
0.7030	patient_protection
0.6975	obamacare_insurance
0.6948	healthcare_law
0.6799	buy_health
0.6786	obamacare_would

Our model:
0.4987	obamacare
0.4914	medicaid_expansion
0.4834	affordable_care
0.4771	insurance
0.3932	health_insurance
0.3723	insurers
0.3712	coverage
0.3680	premiums
0.3599	million_americans
0.3598	individual_mand

In [18]:
cf('leftists')
cf('antifa')

leftists	0.1753, 0.0319, 0.0508, 0.1036, 0.6384, 	176, 32, 51, 104, 641, 
Pretrained:
0.6805	called_liberals
0.6750	liberals
0.6546	america_hating
0.6530	sjws
0.6524	liberals_progressives
0.6519	left_wingers
0.6471	hate_mongers
0.6435	marxists
0.6434	radical_leftists
0.6414	upper_caste

Our model:
0.3382	progressives
0.3083	liberals
0.2701	environmentalists
0.2682	social_justice
0.2608	grassroots
0.2451	support
0.2428	marxist
0.2338	backing
0.2252	fancies
0.2245	ghana

antifa	0.1483, 0.0068, 0.1061, 0.0163, 0.7224, 	109, 5, 78, 12, 531, 
Pretrained:
0.7640	anti-fascist
0.7229	alt_left
0.7210	deray_mckesson
0.7154	blacklives_matter
0.7036	anti-fascists
0.7026	matter_protesters
0.6984	anti-trump_protesters
0.6881	black_bloc
0.6795	white_supremacists
0.6775	jason_kessler

Our model:
0.2746	egyptian_leader
0.2725	far_left
0.2612	much_value
0.2474	never_vote
0.2460	left_wing
0.2443	domestic_extremists
0.2412	anti-fascist
0.2409	new_left
0.2407	protest_movement
0.2406	white_nationalists



In [19]:
cf('guns')

guns	0.4365, 0.0424, 0.2096, 0.0833, 0.2282, 	3109, 302, 1493, 593, 1625, 
Pretrained:
0.7178	firearms
0.6305	gun_enthusiasts
0.6286	semi-automatic_weapons
0.6231	ammo
0.6207	handguns
0.6139	gun_laws
0.6128	buy_guns
0.6080	assault_rifles
0.6066	firearm
0.6060	semiautomatic_weapons

Our model:
0.4915	firearms
0.4871	firearm
0.4635	nra
0.4356	gun_violence
0.4227	ammunition
0.4106	weapons
0.3995	gun_laws
0.3888	gun_control
0.3887	second_amendment
0.3776	handguns



In [20]:
cf('trade')

trade	0.0892, 0.0624, 0.0564, 0.7181, 0.0738, 	1529, 1069, 967, 12302, 1265, 
Pretrained:
0.5864	education_trade
0.5812	infrastructure_corporate
0.5559	actual_trade
0.5520	yen_carry
0.5515	trade_could
0.5504	investment_agreements
0.5465	corporate_regulation
0.5437	investment_flows
0.5435	u.s._goods
0.5382	corporate_governance

Our model:
0.3399	goods
0.3214	economy
0.3185	trading
0.3089	cap
0.3033	so
0.3013	democratic
0.2945	given
0.2935	<NUM>
0.2893	stock
0.2859	for



In [21]:
cf('crooked_hillary')
cf('lyin_ted')
cf('little_marco')

crooked_hillary	0.3624, 0.0336, 0.1946, 0.0671, 0.3423, 	54, 5, 29, 10, 51, 
Pretrained:
0.7717	lyin
0.7677	lyin_ted
0.7644	lying_media
0.7625	nasty_woman
0.7605	little_marco
0.7530	candidate_ever
0.7529	attack_hillary
0.7513	attacking_hillary
0.7499	fucking_moron
0.7467	el_trumpo

Our model:
0.3027	lost_support
0.3022	basketball_game
0.2968	private_e-mail
0.2915	conte
0.2898	even_bill
0.2829	one_post
0.2818	chinese_competition
0.2805	trump_already
0.2729	wrong_path
0.2728	anti-media

lyin_ted	0.2941, 0.0784, 0.4314, 0.0196, 0.1765, 	15, 4, 22, 1, 9, 
Pretrained:
0.8914	little_marco
0.8196	lyin
0.8062	small_hands
0.7978	ted_cruz.
0.7936	trump_calling
0.7881	real_donald
0.7807	cruz_wife
0.7804	snit
0.7803	still_trump
0.7776	typical_politician

Our model:
0.3647	model_suggests
0.3621	well_okay
0.3603	abbi
0.3583	bush_replied
0.3571	telling_cnn
0.3565	josh_mandel
0.3553	jamie_foxx
0.3551	became_friends
0.3549	negatively_impact
0.3547	cavuto_asked

little_marco	0.4848, 0.0909, 0.2424, 0.03

In [22]:
cf('lgbt')

lgbt	0.3891, 0.0632, 0.1926, 0.1031, 0.2519, 	400, 65, 198, 106, 259, 
Pretrained:
0.7207	gay_lesbian
0.6757	bisexual
0.6623	transgender_community
0.6552	lgbt_movement
0.6512	lgbt_community
0.6449	lesbian_rights
0.6414	lesbian_gay
0.6403	transgender
0.6371	transgender_issues
0.6366	bisexual_transgender

Our model:
0.3644	discrimination
0.3287	transgender
0.3223	gay
0.3043	gender_identity
0.2989	trans
0.2881	sexual_orientation
0.2753	sex_marriage
0.2701	advocacy_group
0.2641	homosexuality
0.2612	advocacy

